In [ ]:
import keys

consumer_key = keys.key
consumer_secret = keys.secret
bearer_token = keys.bearer
access_token = keys.access_token
access_secret = keys.access_secret

In [ ]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if api.verify_credentials:
    print("Auth completed successfuly!")
else:
    print("Issue occoured during authentication")

In [ ]:
from serializer import Serializer

se = Serializer("followers")

In [ ]:
accounts = ["mizzaro", "damiano10", "Miccighel_", "eglu10", "KevinRoitero"]
for account in accounts:
    serializer = se.Serializer(f'data/{account}')
    profile = api.get_user(account)
    profile_json = profile._json
    serializer.serialize_json(f'{account}_profile.json', profile_json)